In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv('results/processed.tsv', sep='\t', header=0, index_col=0)
df

In [16]:
df['has_hit'] = 0
df.loc[df['true_positives'] > 0, 'has_hit'] = 1

In [17]:
df_tophit = df.query("top_n == 1 and method=='gpt'")

In [10]:
len(df_tophit.query("true_positives == 0"))

3804

In [11]:
len(df_tophit.query("true_positives > 0"))

1380

In [14]:
df_tophit[df_tophit['true_positives']
df_tophit.df_tophit[df_tophit['true_positives'] > 0, 'has_top_term'] = 1


/var/folders/nc/m4tx21912kv1b8nk3zzx9plr0000gn/T/ipykernel_87313/1580905988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tophit["has_top_term"] = (df_tophit['true_positives'] > 0).astype(int)


In [24]:
df_tophit.groupby(["source", "model"]).mean("has_hit").reset_index()[["source", "model", "has_hit"]]

,source,model,has_hit
0,GO,3.0,0.194444
1,GO,3.5,0.272569
2,GO,4.0,0.371528
3,NONE,3.0,0.144097
4,NONE,3.5,0.364583
5,NONE,4.0,0.315972
6,RefSeq,3.0,0.178819
7,RefSeq,3.5,0.288194
8,RefSeq,4.0,0.265625


In [36]:
df.query("top_n==10 and cutoff==0.05 and closure==True").groupby(["source", "model"]).\
mean("has_hit").reset_index()[["source", "model", "has_hit"]]

,source,model,has_hit
0,GO,3.0,0.614583
1,GO,3.5,0.697917
2,GO,4.0,0.864583
3,NONE,3.0,0.559028
4,NONE,3.5,0.812500
5,NONE,4.0,0.798611
6,RefSeq,3.0,0.517361
7,RefSeq,3.5,0.739583
8,RefSeq,4.0,0.756944


In [46]:
import numpy as np

def himax(tbl):
    """
    Prepare a table for publication, highlighting maximum values
    """
    tbl_subset = tbl.select_dtypes(include=[np.number]).columns
    return tbl.style.hide().highlight_max(subset=tbl_subset, axis=0, props='font-weight:bold').format(precision=3)


In [47]:
def has_hit_summary(top_n=10, cutoff=0.05, closure=True):
    df_subset = df.query(f"top_n=={top_n} and cutoff=={cutoff} and closure=={closure}")
    grp = df_subset.groupby(["source", "model"])
    return grp.mean("has_hit").reset_index()[["source", "model", "has_hit"]]
    

In [48]:
himax(has_hit_summary(10, 0.05, True))

source,model,has_hit
GO,3.000,0.615
GO,3.500,0.698
GO,4.000,0.865
NONE,3.000,0.559
NONE,3.500,0.812
NONE,4.000,0.799
RefSeq,3.000,0.517
RefSeq,3.500,0.740
RefSeq,4.000,0.757


In [49]:
himax(has_hit_summary(10, 0.05, False))

source,model,has_hit
GO,3.000,0.260
GO,3.500,0.410
GO,4.000,0.583
NONE,3.000,0.222
NONE,3.500,0.535
NONE,4.000,0.503
RefSeq,3.000,0.260
RefSeq,3.500,0.462
RefSeq,4.000,0.399
